# Join: embedding, references en joins

In MongoDB kan een document andere documenten bevatten (*embedding*).
Een alternatief is het gebruik van verwijzingen naar andere documenten (*referencing*).
Als verwijzing naar een document gebruik je de `_id` (key) daarvan.

Als je verwijzingen naar andere documenten gebruikt,
heb je `join`s nodig als je de gegevens uit meerdere documenten wilt combineren.
In MongoDB moet je deze joins zelf programmeren, in de toepassing.
MongoDB heeft geen ingebouwde join-operaties, zoals je die in SQL wel hebt.

## Normalisatie in relationele databases

Bij het ontwerpen van een relationele database probeer je deze gewoonlijk te normaliseren.
Het doel van deze normalisatie is om ervoor te zorgen dat alle gegevens in de database "onafhankelijk" zijn, 
anders gezegd, dat een bepaald gegeven maar op één plaats in de database voorkomt.

De belangrijkste reden voor deze normalisatie is dat dit het veranderen van de database op een consistente manier veel eenvoudiger maakt.
Immers, als een gegeven op meerdere plaatsen voor komt, dan moet je bij een verandering al deze verschillende plaatsen bijwerken, om weer een consistente database-toestand te krijgen.

> Voorbeeld: als het adres van een persoon op 3 plaatsen in de database voorkomt, dan moet je bij een verhuizing al deze 3 plaatsen bijwerken. Als je er één of twee vergeet dat is helemaal niet meer duidelijk welk adres nu het juiste is.

Een andere reden is dat deze normalisatie, door het voorkomen van duplicatie, tot een compactere database leidt: het spaart ruimte.
Dit argument is tegenwoordig minder sterk dan vroeger, toen ruimte voor data schaars en duur was.

Normalisatie heeft ook nadelen.
Het belangrijkste nadeel is dat je bij het opvragen van de data uit de database vaak één of meerdere joins nodig hebt:
je moet de waarden uit verschillende tabellen combineren tot het gevraagde resultaat.
Dit leidt bij leesopdrachten (queries) tot extra leesopdrachten voor de verschillende tabellen.

Overigens is de basisopzet van relationele databases sterk gericht op het toewijzen van data aan verschillende tabellen.
Zo moet je meervoudige relaties voorstellen met meerdere tabellen enn met verwijzingen tussen deze tabellen (via *foreign keys*), omdat een rij geen meervoudige waarden kan bevatten.
(Voorbeeld: je kunt niet een array van telefoonnummers in een rij opnemen.)

> Je kunt dit voor kleine arrays misschien oplossen door ruimte voor een paar waarden op te nemen, bijvoorbeeld voor 3 telefoonnummers. Maar dat leidt enerzijds tot beperkingen waar je later tegenaan loopt - als je er 4 nodig hebt, en anderzijds tot extra gaten in de tabel voor die rijen die deze meervoudige ruimte niet gebruiken. 

## Normalisatie in MongoDB

In MongoDB kun je in een document andere documenten opnemen (*embedding*).
Je kunt zelfs arrays van documenten opnemen in een document.

Dit kan leiden tot duplicatie van deeldocumenten, met het genoemde probleem van wijzigen van deze deel-documenten tot gevolg.

In plaats van embedding kun je ook *referenties* naar andere documenten gebruiken (via de `_id` van die documenten, eigenlijk een *foreign key*).

Het gevolg is dat je bij het gebruik mogelijk gegevens uit verschillende *collections* moet combineren.
Bij een SQL-database gebruik je daarvoor een join.
MongoDB heeft *geen ingebouwde mogelijkheden voor joins*:
als applicatieprogrammeur moet je deze referenties zelf oplossen, door een query met de `_id` in de betreffende collectie.

Net als bij het gebruik van schema's kun je als ontwerper van de database zelf bepalen waar je de grens legt tussen het gebruik van embedding en het gebruik van referenties.

## Voorbeeld: agenda en contacten

In een agenda-document staan (in de voorbeelden) de email-adressen van de deelnemers (`participants`) van de afspraak.
Deze kun je zien als (tijdelijke) keys van de betrokken contacten.

Deze email-adressen kunnen we vervangen door de `_id`s van deze contacten:
we gebruiken dan verwijzingen (*referencess*) tussen documenten, in plaats van *embedding*.
In zekere zin is de relatie tussen de agenda-documenten en de contacts-documenten genormaliseerd:
er is geen duplicatie van gegevens tussen deze documenten.

Zie ook: https://docs.mongodb.com/manual/core/data-modeling-introduction/

Het gebruik van verwijzingen (*references*) betekent dat we *joins* nodige hebben als we de gegevens van verschillende documenten willen combineren.
Bijvoorbeeld: als we een lijst van afspraken willen maken met daarin de namen van de deelnemers, dan moeten we daarvoor de bijbehorenden contact-documenten raadplegen.
Dit moeten we in de toepassing programmeren (met alle risico's op fouten e.d.).

> In een SQL-database gebruik je hiervoor een *JOIN*: naast deze query hoe je daarvoor in de  toepassing niets te programmeren.

In [ ]:
import os
import re
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
import pymongo

pd.set_option('max_colwidth',160)

userline = !echo $USER
username = userline[0]
dbname = username + "-demodb"
print("Database name: " + dbname)

print('Mongo version', pymongo.__version__)
client = pymongo.MongoClient('localhost', 27017)
db = client[dbname]
contacts = db.contacts
agenda = db.agenda

mongopathfile = !cat mongopath
mongopath = mongopathfile[0]

In [ ]:
contacts.drop()
os.system(mongopath + 'mongoimport -d ' + dbname + ' -c contacts adressen.json')

In [ ]:
agenda.drop()
os.system(mongopath + 'mongoimport -d ' + dbname + ' -c agenda agenda.json')

Merk op dat een agenda-items een lijst van deelnemers (`participants`) bevat.
In de invoer identificeren we deze deelnemers met hun email-adres.

In [ ]:
list(agenda.find())

## Maken van *references*

In de agenda-items in de invoer gebruiken we het email-adres als de identificatie ("key") voor de deelnemers.
Voor de invoer is dit acceptabel, maar niet voor lange-termijn opslag in de database:
het email-adres van een persoon kan veranderen.
Ook de naam kun je niet als key gebruiken: namen zijn niet voldoende uniek.

Daarom willen we bij het maken/toevoegen van een afspraak het email-adres als identificatie vervangen door de interne identificatie van het contact-document: de `_id`-key.
We krijgen dan een genormaliseerde vorm gebaseerd op *references* in plaats van *embedding*.

NB: bij de update weten we hier zeker dat het document (record) al bestaat: we hoeven hier geen `$upsert` te gebruiken.

De onderstaande functie `connect` zoekt voor de deelnemers (`participants`) van het agenda-document het bijbehorende contact-document, aan de hand van het email-adres.
Vervolgens wordt als `id` veld van de deelnemer de *key* (`_id`) van het contact-document ingevuld.
Het agenda-document in de database wordt met deze gegevens bijgewerkt.

In [ ]:
def connect (agendaItem):
    participants = agendaItem["participants"]
    for person in participants:
        pList = list(contacts.find({"email": person["email"]}))
        if len(pList) == 1:
            person["id"] = pList[0]["_id"]        
        else:
            print(person["email"])
            print("error in connect")
    agenda.update_one({"_id": agendaItem["_id"]}, {"$set": {"participants": participants}})

We werken alle agenda-documenten bij met behulp van `connect`: de `participants` in de agenda-documenten verwijzen dan naar de bijbehorende contact-documenten.

In [ ]:
aList = list(agenda.find())
for item in aList:
    connect(item)

In het onderstaande overzicht zie je deze referenties bij de `participants`:

In [ ]:
list(agenda.find())

## Join

Als we de gegevens van een agenda-document willen combineren met gegevens uit de contact-documenten, bijvoorbeeld de namen van de deelnemers, dan hebben we een *join* nodig.

In MongoDB moeten we deze join zelf programmeren.
Een join bestaat uit twee elementen:

1. het zoeken van de bijbehorende documenten (`find`);
2. het toevoegen van waarden uit een opgezocht bijbehorend document aan het resultaat-document (bijvoorbeeld, de naam uit het contact-document).

Gegeven een lijst van agenda-items, voeg hier de namen van de deelnemers aan toe.

De onderstaande functie `addNames` zoekt aan de hand van de keys (`id`) de contact-documenten van de deelnemers, en voegt de naam van elke deelnemer toe aan het agenda-document.

In [ ]:
def addNames(agendaItem):
    participants = agendaItem["participants"]
    for person in participants:
        pList = list(contacts.find({"_id": person["id"]})) ## (1) find
        if len(pList) == 1:
            person["name"] = pList[0]["name"]  ## (2) toevoegen van naam   
        else:
            print(person["email"])
            print("error in addName")

We voeren hier de genoemde *join* uit voor alle agenda-documenten.
In de praktijk voer je eerst een selectie uit op de agenda-documenten die gebruikt worden;
alleen voor die documenten voer je dan de join uit.

In [ ]:
aList = list(agenda.find())
for item in aList:
    addNames(item)
aList

**Opdracht**
Maak een lijst van agenda-documenten voor de "Beleidsplan"-bijeenkomsten, met daarin als onderdeel van de `participants` de namen en woonplaatsen van de deelnemers.

Splits dit in 3 stappen:

1. definitie van een functie `joinContacts(agendaItem)`
2. construeren van de lijst met "Beleidsplan"-bijeenkomsten
3. het toepassen van de genoemde functie op alle elementen in deze lijst.

### Opmerkingen

#### Meer gegevens per *participant*

De gegevens van de deelnemers in de afspraak kunnen we later uitbreiden, bijvoorbeeld met de opmerking of ze al uitgenodigd zijn, en of ze al bevestigd hebben.
Dat valt buiten het bestek van dit voorbeeld.

#### Gedeeltelijke de-normalisatie

In het voorbeeld hierboven kun je de namen van de deelnemers ook in het agenda-document opnemen, naast de `id` van het contact-document.
Op die manier hoef je niet steeds een join uit te voeren met de contact-documenten als je alleen de namen vn de deelnemers nodig hebt.
Er is dan wel sprake van *duplicatie* van gegevens: de naam van een contact komt zowel in het contact-document als in meerdere agenda-documenten voor.
Als de naam van een contact niet verandert, levert dit geen probleem met inconsistenties op.
Een klein nadeel is dat de opslag wat minder compact is, maar dat weegt niet op tegen het voordeel van snellere toegang tot de gegevens

> Duplicatie van gegevens versnelt in het algemeen het opzoeken van gegevens,
  maar kan het aanpassen van gegegevens (veel) lastiger maken.
  Voor gegevens die niet veranderen is duplicatie geen probleem.

#### Caching

De aanpak in de functie `addNames` is niet de meest efficiënte:
eenzelfde contact-verwijzing komt meerdere malen voor, we zoeken dit document steeds opnieuw op.
Dit kunnen we efficiënter maken door een *cache* toe te voegen van personen die we al eerder opgezocht hebben.
Dat vraagt maar een kleine aanpassing in het algoritme.

> Toevoegen: voorbeeld van caching

#### Linked data

In het voorbeeld van de agenda gebruiken we het email-adres als identificatie van een persoon, of, later, de interne key van het contact-document.
Bij Linked Data zullen we nog andere manieren tegenkomen voor het identificeren van een persoon (contact).

#### Embedding versus referencing

Bij MongoDB kun je kiezen tussen embedding en referencing.
Wat zijn hierbij goede criteria:

* gebruik altijd *referencing* voor documenten die ook zelfstandig voor kunnen komen, zoals in het bovenstaande voorbeeld agenda-documenten en contact-documenten.
    * eventueel kun je deze referencing uitbreiden met een gedeeltelijke embedding van niet-veranderende velden (zoals de naam).
* gebruik *embedding* voor gegevens die bij elkaar horen, en die zelfstandig geen betekenis hebben. In het bovenstaande voorbeeld is het veld `participants` daarvan een voorbeeld.
